In [1]:
import os.path, sys
sys.path.append(os.path.join(os.path.dirname(os.path.realpath('__file__')), os.pardir))
import reader
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
import re
import pandas as pd
from nltk import ngrams, word_tokenize

In [2]:
data=reader.Data()

Local data read/write folder path:
	Default path: E:\anil\IIT Sop\Term02\MATH497\MATH497-DiabeticRetinopathy/data/

Data: systemic_disease_list 
File: systemic_disease_list.pickle
File already exists.

Data: SNOMED_problem_list 
File: SNOMED_problem_list.pickle
File already exists.

Data: macula_findings_for_Enc 
File: macula_findings_for_Enc.pickle
File already exists.

Data: SL_Lens_for_Enc 
File: SL_Lens_for_Enc.pickle
File already exists.

Data: family_hist_list 
File: family_hist_list.pickle
File already exists.

Data: systemic_disease_for_Enc 
File: systemic_disease_for_Enc.pickle
File already exists.

Data: all_person_data 
File: all_person_data_Dan_20170330.pickle
File already exists.

Data: family_hist_for_Enc 
File: family_hist_for_Enc.pickle
File already exists.

Data: person_profile 
File: person_profile_df.pickle
File already exists.

Data: all_encounter_data 
File: all_encounter_data_Dan_20170330.pickle
File already exists.

Data: encounters 
File: encounters.pickle
File 

In [3]:
table_name = 'macula_findings_for_Enc'
df = data[table_name].copy()
finding_columns = [col for col in list(df.columns.values) if '_Finding' in col]
modifier_columns = [col for col in list(df.columns.values) if '_Modifer' in col]

In [4]:
from collections import Counter
all_finding_text_counter = Counter()
for col in finding_columns:
    all_finding_text_counter.update(dict(df[col].value_counts()))

all_finding_text_counter.most_common()

[('flat no hemorrhages exudates or pigmentary changes', 27353),
 ('flat no hemorrhages exudates pigmentary changes or no macular edema',
  21184),
 ('(-)CSME', 2920),
 ('no macular edema', 2817),
 ('(-) CSME', 2087),
 ('no hemorrhage or exudates', 982),
 ('no CSME', 934),
 ('-CSME', 864),
 ('RPE mottling', 857),
 ('non-proliferative diabetic retinopathy', 772),
 ('flat no hemorrhages exudates pigmentary changes or no macular edema (-)CSME',
  642),
 ('flat no hemorrhages exudates pigmentary changes (-) CSME', 557),
 ('flat no hemorrhages exudates pigmentary changes (-)CSME', 528),
 ('flat no hemorrhages exudates or pigmentary changes -CSME', 436),
 ('ERM', 308),
 ('----- (-) EDEMA -----', 296),
 ('edema', 286),
 ('-----NORMAL-----', 213),
 ('diabetic retinopathy', 197),
 ('flat no hemorrhages exudates or pigmentary changes(-)CSME', 189),
 ('flat no hemorrhages exudates pigmentary changes or no macular edema -CSME',
  187),
 ('epiretinal membrane formation', 180),
 ('not viewed', 176),


In [5]:
symbol_pattern = r'<sym>\s?[\-\+]\s?'
comp_pattern = re.compile(symbol_pattern)
comp_pattern2 = re.compile(r'(\(\s*\()')
comp_pattern3 = re.compile(r'(\)\s*\))')
comp_pattern4 = re.compile(r'(\-{2,})')
comp_pattern5 = re.compile(r'(\_+)')

def find_sublists(seq, sublist):
    length = len(sublist)
    for index, value in enumerate(seq):
        if value == sublist[0] and seq[index:index+length] == sublist:
            yield index, index+length

import itertools

def replace_sublist(seq, target, replacement, maxreplace=None):
    sublists = find_sublists(seq, target)
    if maxreplace:
        sublists = itertools.islice(sublists, maxreplace)
    for start, end in sublists:
        seq[start:end] = replacement           

class desc_tokens_streamer(object):
    def __init__(self, descriptions_counter):
        self.descriptions_counter  = Counter(descriptions_counter)
        self.modified_tokens = {}
        self.descriptions_tokens = {}
        self.repeat_by_count = False
    
    def add_descriptions(self, descriptions_counter):
        self.descriptions_counter.update(descriptions_counter)
    
    def clean_desc(self, desription):
        new_desc = desription.lower()
        new_desc = comp_pattern4.sub(' ',new_desc)
        new_desc = comp_pattern5.sub(' ',new_desc)
        
        characters_to_remove = ['/', ';', ',', '.']
        for character in characters_to_remove:
            if character in new_desc:
                new_desc = new_desc.replace(character, u' ')
            
        
        if u'-' in new_desc:
            new_desc = new_desc.replace(u'-', u' ( - ) ')
#             new_desc = ' '.join(new_desc.split())
        
        if u'+' in new_desc:
            new_desc = new_desc.replace(u'+', u' ( + ) ')
#             new_desc = ' '.join(new_desc.split())
        
#         new_desc = comp_pattern.sub('(\g<sym>)', new_desc)
#         new_desc = new_desc.replace('( (', '(').replace('((', '(').replace('))', ')').replace(') )', ')')
        
        new_desc = comp_pattern2.sub('(', new_desc)
        new_desc = comp_pattern3.sub(')', new_desc)
        
        return new_desc
    
    def __iter__(self):
        for desc_index, (desc, count) in  enumerate(self.descriptions_counter.most_common()):
            repeat_count = 1
            if self.repeat_by_count:
                repeat_count= count
            for i in range(repeat_count):
                yield self.__getitem__(desc)
    
    def __getitem__(self, desc):
        if desc not in self.descriptions_counter:
            self.descriptions_counter.update([desc])
            
        if desc not in self.modified_tokens: 
            self.modified_tokens[desc] = self.clean_desc(desc)
            if self.modified_tokens[desc] not in self.descriptions_tokens:
                word_tokens = word_tokenize(self.modified_tokens[desc])
                replace_sublist(word_tokens,['(', '+', ')'], ['(+)'])
                replace_sublist(word_tokens,['(', '-', ')'], ['(-)'])
#                 replace_sublist(word_tokens,[u'no', '(-)'], ['non-'])
                self.descriptions_tokens[self.modified_tokens[desc]] = word_tokens
        return desc, self.descriptions_tokens[self.modified_tokens[desc]], self.descriptions_counter[desc]

                
macula_desc_tokens = desc_tokens_streamer(all_finding_text_counter)

# i =0
# for desc in macula_desc_tokens:
#     print(desc)
#     i+=1
#     if i>10:break

In [6]:
ngram_counter = Counter()
ngram_dict = {}

num_desc = len(all_finding_text_counter)

for desc_index, (desc, word_tokens, count) in  enumerate(macula_desc_tokens):
    for ngram_length in range(1, len(word_tokens)+1):
        for ngram_tokens in ngrams(word_tokens , ngram_length):
            ngram = ' '.join(ngram_tokens)
            ngram_dict.setdefault(ngram_length, set()).add(ngram)
            ngram_counter.update({ngram:count})
    if (desc_index+1)%(int(num_desc/10)) == 0:
        print('%s descriptions tokenized and n-gramed.' % (desc_index+1))
print('%s descriptions tokenized and n-gramed.' % (desc_index+1))


print("There are %s ngrams in total:" % (sum(len(ngram_dict[n]) for n in ngram_dict.keys())))
for n in ngram_dict.keys():
    print("\t%s, %s-gram tokens." %(len(ngram_dict[n]), n))

1560 descriptions tokenized and n-gramed.
3120 descriptions tokenized and n-gramed.
4680 descriptions tokenized and n-gramed.
6240 descriptions tokenized and n-gramed.
7800 descriptions tokenized and n-gramed.
9360 descriptions tokenized and n-gramed.
10920 descriptions tokenized and n-gramed.
12480 descriptions tokenized and n-gramed.
14040 descriptions tokenized and n-gramed.
15600 descriptions tokenized and n-gramed.
15602 descriptions tokenized and n-gramed.
There are 183290 ngrams in total:
	2256, 1-gram tokens.
	15970, 2-gram tokens.
	30911, 3-gram tokens.
	34582, 4-gram tokens.
	30624, 5-gram tokens.
	23702, 6-gram tokens.
	16868, 7-gram tokens.
	11317, 8-gram tokens.
	7246, 9-gram tokens.
	4408, 10-gram tokens.
	2540, 11-gram tokens.
	1414, 12-gram tokens.
	734, 13-gram tokens.
	363, 14-gram tokens.
	186, 15-gram tokens.
	89, 16-gram tokens.
	44, 17-gram tokens.
	21, 18-gram tokens.
	8, 19-gram tokens.
	4, 20-gram tokens.
	2, 21-gram tokens.
	1, 22-gram tokens.


In [7]:
removed_ngrams_dict = {}
newly_removed = []
for n in ngram_dict.keys():
    for ngram in ngram_dict[n]:
        if ngram not in removed_ngrams_dict.get(n, set()):
            if '(' in ngram or ')' in ngram:
                if ngram.count('(') != ngram.count(')'):
                    removed_ngrams_dict.setdefault(n, set()).add(ngram)
                    newly_removed.append(ngram)


print('%s ngrams removed' % (len(newly_removed)))



5031 ngrams removed


In [8]:
newly_removed[:15]

[u'(',
 u')',
 ') neg',
 u'1dd )',
 u'( pdr',
 '( greater',
 u'( due',
 u'( poor',
 u'moderate )',
 u'trauma )',
 u') cme',
 u'( exudates',
 'pp (',
 u'maculae (',
 u'4 )']

In [9]:
parent_ngram_dict = {}
ngram_index = 0
for n in ngram_dict.keys()[:-1]:
    print('Processing %s-grams' % (n))
    for ngram in ngram_dict[n]:
        if ngram not in removed_ngrams_dict.get(n, set()):
            for bigger_ngram in ngram_dict[n+1]:
                if ngram in bigger_ngram:
                    if ngram_counter[ngram]<= ngram_counter[bigger_ngram]:
                        removed_ngrams_dict.setdefault(n, set()).add(ngram)
                        parent_ngram_dict[ngram] = bigger_ngram
                        break
        ngram_index+=1
        if (ngram_index+1)%(10**4) == 0:
            print('\t%s ngrams processed for parent ngram.' % (ngram_index+1))
print('%s ngrams processed for parent ngram.' % (ngram_index+1))

            

Processing 1-grams
Processing 2-grams
	10000 ngrams processed for parent ngram.
Processing 3-grams
	20000 ngrams processed for parent ngram.
	30000 ngrams processed for parent ngram.
	40000 ngrams processed for parent ngram.
Processing 4-grams
	50000 ngrams processed for parent ngram.
	60000 ngrams processed for parent ngram.
	70000 ngrams processed for parent ngram.
	80000 ngrams processed for parent ngram.
Processing 5-grams
	90000 ngrams processed for parent ngram.
	100000 ngrams processed for parent ngram.
	110000 ngrams processed for parent ngram.
Processing 6-grams
	120000 ngrams processed for parent ngram.
	130000 ngrams processed for parent ngram.
Processing 7-grams
	140000 ngrams processed for parent ngram.
	150000 ngrams processed for parent ngram.
Processing 8-grams
	160000 ngrams processed for parent ngram.
Processing 9-grams
	170000 ngrams processed for parent ngram.
Processing 10-grams
Processing 11-grams
	180000 ngrams processed for parent ngram.
Processing 12-grams
Proc

In [13]:
print('%s ngrams (%s%%) with parents removed from %s total ngrams' % (len(parent_ngram_dict), len(parent_ngram_dict)*100/(1+ngram_index),  ngram_index+1))
print('%s ngrams remain' % (sum([len(ngram_list) for ngram_list in ngram_dict.values()] )-sum([len(ngram_list) for ngram_list in removed_ngrams_dict.values()] )))


151892 ngrams (82%) with parents removed from 183290 total ngrams
26367 ngrams remain


In [14]:
selected_ngrams_dict = {}
for n in ngram_dict.keys():
    selected_ngrams_dict[n] = ngram_dict[n]
    if n in removed_ngrams_dict:
        selected_ngrams_dict[n] = selected_ngrams_dict[n] - removed_ngrams_dict[n]
    else:
        selected_ngrams_dict[n] = selected_ngrams_dict[n]

print("There are %s ngrams in total:" % (sum(len(selected_ngrams_dict[n]) for n in selected_ngrams_dict.keys())))
for n in selected_ngrams_dict.keys():
    print("\t%s, %s-gram tokens." %(len(selected_ngrams_dict[n]), n))

There are 26367 ngrams in total:
	894, 1-gram tokens.
	4359, 2-gram tokens.
	5226, 3-gram tokens.
	4242, 4-gram tokens.
	3040, 5-gram tokens.
	2293, 6-gram tokens.
	1798, 7-gram tokens.
	1476, 8-gram tokens.
	1080, 9-gram tokens.
	784, 10-gram tokens.
	496, 11-gram tokens.
	313, 12-gram tokens.
	190, 13-gram tokens.
	80, 14-gram tokens.
	51, 15-gram tokens.
	22, 16-gram tokens.
	10, 17-gram tokens.
	9, 18-gram tokens.
	2, 19-gram tokens.
	1, 20-gram tokens.
	0, 21-gram tokens.
	1, 22-gram tokens.


In [21]:
list(x for x in ngram_dict[2] if '(-)' in x)

[u'1 (-)',
 u'(-) extensive',
 u'(-) scattered',
 u'longstanding (-)',
 u'erm (-)',
 u'on (-)',
 u'(-) aneurysm',
 u'(-) vascular',
 u'(-) csme',
 u'(-) csmd',
 u'(-) hypertensive',
 u'streaks (-)',
 u'mottling (-)',
 u'super (-)',
 u'ripple (-)',
 u'parafoveally (-)',
 u'proliferative (-)',
 u'motteling (-)',
 u'vmt (-)',
 u'(-) patient',
 u': (-)',
 u'chg (-)',
 u'para (-)',
 u'(-) stage',
 u'tact (-)',
 u'nasally (-)',
 u'4 (-)',
 u'ma (-)',
 u'mass (-)',
 u'(-) small',
 u'view (-)',
 u'watski (-)',
 u'cnv (-)',
 u'(-) hemorrhages',
 u'(-) diaret',
 u'fov (-)',
 u'microns (-)',
 u'(-) very',
 u'(-) nv',
 u'(-) no',
 u'2 (-)',
 u'(-) cmehemes',
 u'(-) none',
 u'(-) flat',
 u'change (-)',
 u'(-) csmeflat',
 u'(-) resolution',
 u'flecks (-)',
 u'cl (-)',
 u'mas (-)',
 u'b (-)',
 u'hems (-)',
 u'retina (-)',
 u'ppa (-)',
 u'~8 (-)',
 u'crack (-)',
 u'macroaneurysms (-)',
 u'(-) closed',
 u'(-) flattening',
 u'(-) edematous',
 u'refelx (-)',
 u'(-) pigm',
 u'(-) not',
 u'(-) excessive',


In [27]:
bad_suffix_ngrams = set()
multiple_prefix_ngrams = set()
for n in selected_ngrams_dict.keys():
    for ngram in selected_ngrams_dict[n]:
        if ngram[-3:] == '(-)' or ngram[-3:] == '(+)':
            bad_suffix_ngrams.add(ngram)
        
        if '(-)' in ngram or '(+)' in ngram:
            if sum([ngram.count('(-)'), ngram.count('(+)')]) > 1: 
                multiple_prefix_ngrams.add(ngram)
            elif ngram[:3] != '(-)' or ngram[:3] != '(+)':
                bad_suffix_ngrams.add(ngram)
                
len(bad_suffix_ngrams) + len(multiple_prefix_ngrams)

7877

In [121]:
# from nltk import ngrams, word_tokenize

# tokenized_descriptions = {}

# num_desc = len(all_finding_text_counter)

# for desc_index, (desc, count) in  enumerate(all_finding_text_counter.most_common()):
#     new_desc = desc
#     if u'-' in desc:
#         new_desc = desc.replace('-', ' - ')
#     word_tokens = word_tokenize(new_desc)
#     tokenized_descriptions[desc] = word_tokens

In [104]:
# new_desc = '-----EDEMA'
# macula_desc_tokens[new_desc]

['edema']

In [12]:
# onegram_tokens = Counter()
# macula_desc_tokens.repeat_by_count=True
# for desc in macula_desc_tokens:
#     onegram_tokens.update(desc)

# for word, count in onegram_tokens.most_common():
#     if len(word) > 10 and count<30:
#         print(word, count)

TypeError: unhashable type: 'list'